In [1]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

2024-12-02 20:54:50.266873: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
# In case of errors, check if you are using the correct versions of the libraries. This notebook works on the following versions of libraries. Specify these versions if you get version related errors.
# TensorFlow version: 2.17.0
# Pandas version: 2.1.4
# NumPy version: 1.26.4
# Scikit-learn version: 1.5.2
# Keras version: 3.4.1

# print("TensorFlow version:", tf.__version__)
# print("Pandas version:", pd.__version__)
# print("NumPy version:", np.__version__)
# print("Scikit-learn version:", sklearn.__version__)
# print("Keras version:", tf.keras.__version__)

## Part 1: Import the Housing data and do feature transformations

In [3]:
df= pd.read_csv('house_price_full+(1).csv')
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [4]:
X = df.copy()
# Remove target
Y = X.pop('price')

# perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# perform log transformation of target variable (For Sandeep: Is this needed?)
Y = np.log(Y)

In [4]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [5]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

## Part 2: Create Model Using `keras`

![](multiple_neurons.png)

In [6]:
from tensorflow import keras

In [7]:
# model = keras.Sequential(
#     [
#         keras.layers.Dense(
#             2, activation="sigmoid", input_shape=(X.shape[-1],)
#         ),
#         keras.layers.Dense(1, activation="linear")
#     ]
# )
# model.summary()

# Newer versions of Keras support this:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(X.shape[-1],)))
model.add(keras.layers.Dense(2, activation="sigmoid"))
model.add(keras.layers.Dense(1, activation="linear"))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 2)              │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9 (36.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)

```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [10]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1)
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [11]:
model.fit(X,Y.values,epochs=10,batch_size=32)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 139.9492  
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 42.5699 
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.7756  
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.5978 
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4069 
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1959 
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1722 
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1479 
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1562 
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1503 


In [12]:
model.predict(X)[:,0]

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([12.834526 , 13.51856  , 13.098537 , 13.125709 , 12.976137 ,
       12.721556 , 12.928034 , 13.30406  , 13.273609 , 12.7550335,
       13.007362 , 13.375124 , 13.2420635, 12.704786 , 13.253771 ,
       12.763072 , 12.944794 , 13.432968 , 12.849421 , 12.752611 ,
       13.212221 , 12.715784 , 13.024591 , 13.311159 , 12.799126 ,
       13.159349 , 13.251703 , 13.098712 , 13.214884 , 13.323465 ,
       12.777185 , 13.355599 , 12.915709 , 13.342239 , 13.562759 ,
       13.472897 , 12.682518 , 13.066399 , 13.110291 , 13.261192 ,
       12.949379 , 12.732679 , 12.707045 , 13.033098 , 12.870693 ,
       12.742088 , 13.114172 , 13.098537 , 12.799193 , 13.4876   ,
       13.30406  , 13.40292  , 13.353296 , 13.045722 , 12.914663 ,
       12.966229 , 13.094217 , 13.521492 , 12.8245125, 13.032131 ,
       13.4876   , 13.345345 , 13.311159 , 13.3064   , 13.375124 ,
       12.731144 , 12.980978 , 12.661289 , 13.309525 , 13.198507 ,
       13.236948 , 13.388638 , 13.134031 , 13.08662  , 12.8295